# NHS Written Complaints Against GPs and Dental Practices - Annual Reports

The parent data collection is the [General Practice (including Dental) Complaints Collection (KO41b)](http://digital.nhs.uk/datacollections/ko41b).

The data finder query is for national statisics on [Patient experience / Complaints](http://digital.nhs.uk/searchcatalogue?topics=1/Patient%20experience/Complaints&sort=Relevance&size=10&page=1#top). This query also returns links to experimental quarterly reports.

The data is also registered with data.gov.uk at https://data.gov.uk/dataset/data_on_written_complaints_in_the_nhs

NOTE: *The column names used are not necessarily consistent year on year so some cleaning may be required to get the report generator to work correctly.*

## Setup

Import necessary libraries and supporting administrative data.

In [1]:
import pandas as pd

We can import GP administrative data from the `nhsadmin.sqlite` database as generated in the [NHS and GP Administrative Data.ipynb](NHS%20and%20GP%20Administrative%20Data.ipynb) notebook.


In [94]:
import sqlite3
con = sqlite3.connect("nhsadmin.sqlite")

gp = pd.read_sql_query('SELECT * from epraccur', con)
gp.head()

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,...,Close Date,Status Code,Organisation Sub-Type code,Commissioner,Join Provider/Purchaser Date,Left Provider/Purchaser Date,Contact Telephone Number,Amended Record Indicator,Provider/Purchaser,Prescribing Setting
0,A81001,THE DENSHAM SURGERY,Y54,Q74,THE HEALTH CENTRE,LAWSON STREET,STOCKTON-ON-TEES,CLEVELAND,None,TS18 1HU,...,None,A,B,00K,2013-04-01 00:00:00,None,01642 672351,0,00K,4
1,A81002,QUEENS PARK MEDICAL CENTRE,Y54,Q74,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,None,TS18 2AW,...,None,A,B,00K,2013-04-01 00:00:00,None,01642 679681,0,00K,4
2,A81003,VICTORIA MEDICAL PRACTICE,Y54,Q74,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,None,TS26 8DB,...,None,A,B,00K,2013-04-01 00:00:00,None,01429 272945,0,00K,4
3,A81004,WOODLANDS ROAD SURGERY,Y54,Q74,6 WOODLANDS ROAD,None,MIDDLESBROUGH,CLEVELAND,None,TS1 3BE,...,None,A,B,00M,2013-04-01 00:00:00,None,01642 247982,0,00M,4
4,A81005,SPRINGWOOD SURGERY,Y54,Q74,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,None,None,TS14 7DJ,...,None,A,B,00M,2013-04-01 00:00:00,None,01287 619611,0,00M,4


In [95]:
dentists=pd.read_sql_query('SELECT * from egdpprac', con)
dentists.head()

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,Open Date,Close Date,Status Code,Organisation Sub-Type Code,Parent Organisation Code,Join Parent Date,Left Parent Date,Amended Record Indicator
0,V00002,DENTAL SURGERY,Y52,Q37,DENTAL SURGERY,22 MARTYRS AVENUE,CRAWLEY,WEST SUSSEX,None,RH11 7RZ,2008-04-01 00:00:00,2009-03-31 00:00:00,C,D,5P6,2008-04-01 00:00:00,2009-03-31 00:00:00,0
1,V00003,CRABTREE ROAD DENTAL PRACTICE,Y57,Q81,CRABTREE ROAD DENTAL PRACTICE,25 CRABTREE ROAD,CRAWLEY,WEST SUSSEX,None,RH11 7HL,2006-04-01 00:00:00,None,A,D,14G,2015-04-01 00:00:00,None,0
2,V00004,DENTAL SURGERY,Y57,Q81,DENTAL SURGERY,WOODCROFT,BEPTON ROAD,MIDHURST,WEST SUSSEX,GU29 9HH,2006-04-01 00:00:00,None,A,D,14G,2015-04-01 00:00:00,None,0
3,V00005,DENTAL SURGERY,Y57,Q81,DENTAL SURGERY,31 WICK STREET,WICK,LITTLEHAMPTON,WEST SUSSEX,BN17 7JJ,2006-04-01 00:00:00,None,A,D,14G,2015-04-01 00:00:00,None,0
4,V00006,DENTAL SURGERY,Y57,Q81,DENTAL SURGERY,43 BUSTICLE LANE,SOMPTING,LANCING,WEST SUSSEX,BN15 0DJ,2006-04-01 00:00:00,None,A,D,14G,2015-04-01 00:00:00,None,0


In [96]:
#Create a function to grab a zip file from an online location and then grab a specified file from inside it
import requests, zipfile, StringIO
def zipgrabber(url):
    r = requests.get(url)
    z = zipfile.ZipFile(StringIO.StringIO(r.content))
    return z

def csvfileopener(z,k,metadata=False):
    f=None
    for n in z.namelist():
        if n.split('.')[-1]=='csv' and k in n: f=n
    if f is not None:
        df = pd.read_csv(z.open(f))
        df.dropna(how='all',axis=1,inplace=True)
        df=df[[c for c in df.columns if not c.startswith('Unnamed')]]
        if metadata:
            complaints_meta = get_metadata(z)
            cols=['::'.join(x) for x in zip(complaints_meta['Name'],complaints_meta['meta'])]
            df.columns=cols
        return df
    return pd.DataFrame()

In [97]:
def get_metadata(z):
    for n in z.namelist():
        if 'xls' in n and 'meta' in n: f=n
    xl=pd.ExcelFile(z.open(f))
    for n in xl.sheet_names:
        if 'KO41b' in n: sheetname=n
    complaints_meta=xl.parse(sheetname,header=None)
    complaints_meta.columns=['Name','typ','meta']
    complaints_meta = complaints_meta[complaints_meta['Name']!='Column']
    return complaints_meta

## Complaints Data - 2015 / 16

via: http://digital.nhs.uk/datacollections/ko41b
specifically: http://digital.nhs.uk/searchcatalogue?productid=21728

In [98]:
url='http://www.digital.nhs.uk/catalogue/PUB21533/data-writ-comp-nhs-2015-2016-csv.zip'
zip_2015_16=zipgrabber(url)

complaints_2015_16=csvfileopener(zip_2015_16,'KO41a')
complaints_2015_16.head(3)

,Quarter,OrganisationCode,OrganisationName,SiteCode,SiteName,ComplaintsTotalBroughtForward,ComplaintsTotalNew,ComplaintsTotalResolved,ComplaintsNumberUpheld,ComplaintsNumberPartiallyUpheld,...,ProfessionAmbulance,ProfessionSupportToClinicalStaff,ProfessionNHSInfrastructureSupport,ProfessionOther,ProfessionTotal,NHSEnglandRegionCode,NHSEnglandRegionName,NHSEnglandRegionGeographyCode,NHSEnglandRegionGeographyName,OrganisationType
0,201516 Q1,8EL02,Health First (UK) Ltd,8EL02,Health First (UK) Ltd,0,0,0,0,0,...,0,0,0,0,0,QZZ,Special Health Authorities and other statutory...,QZZ,Special Health Authorities and other statutory...,Others
1,201516 Q1,AAW,Allied Healthcare Group,AAW,ALLIED HEALTHCARE GROUP,0,24,21,4,6,...,0,4,0,10,24,QZZ,Special Health Authorities and other statutory...,QZZ,Special Health Authorities and other statutory...,Community Provider Trust
2,201516 Q1,NLO,Vocare,NLO01,NORTHERN DOCTORS URGENT CARE,0,27,27,12,5,...,0,0,0,0,27,QZZ,Special Health Authorities and other statutory...,QZZ,Special Health Authorities and other statutory...,Community Provider Trust


In [99]:
complaints_b_2015_16=csvfileopener(zip_2015_16,'KO41b',True)
complaints_b_2015_16.head(3)

,Year::Year of collection,"Return::Organisation type (NHS England, GP, Dental) making return",NHSERegionCode::NHS England Region Code,NHSERegionName::NHS England Region Name,NHSERegionGeographyCode::NHS England Region Geography Code,NHSERegionGeographyName::NHS England Region Geography Name,Practice_Code::Dental/GP Practice Code,Medical TOTAL::Complaints by area,Dental TOTAL::Complaints by area,Practice admin TOTAL::Complaints by area,...,Practice admin UPHELD::Complaints upheld by area,Other UPHELD::Complaints upheld by area,Total UPHELD::Complaints upheld by area,Communications/Attitude SUBJECT UPHELD::Complaints upheld by subject,Premises SUBJECT UPHELD::Complaints upheld by subject,Practice/Surgery Management SUBJECT UPHELD::Complaints upheld by subject,Practice admin SUBJECT UPHELD::Complaints upheld by subject,Clinical SUBJECT UPHELD::Complaints upheld by subject,Other SUBJECT UPHELD::Complaints upheld by subject,Total SUBJECT UPHELD::Complaints upheld by subject
0,2015-16,NHS England,Y57,South of England,Q70,NHS England South (Wessex),Q70,95,0,1,...,0,0,64,9,0,11,1,42,1,64
1,2015-16,NHS England,Y57,South of England,Q70,NHS England South (Wessex),Q70,0,54,0,...,0,0,29,4,0,5,3,16,1,29
2,2015-16,NHS England,Y56,London,Q71,NHS England London,Q71,460,0,76,...,27,0,185,48,1,56,21,59,0,185


In [103]:
#join complaints parent codes
parentGP=gp[['Organisation Code','Commissioner']]
parentGP.columns=['Organisation Code','Parent Organisation Code']
parentDentist=dentists[['Organisation Code','Parent Organisation Code']]
codeparent=pd.concat([parentGP,parentDentist])
codeparent.head(3)

,Organisation Code,Parent Organisation Code
0,A81001,00K
1,A81002,00K
2,A81003,00K


In [104]:
iwcodes=codeparent[(codeparent['Parent Organisation Code']=='10L')]
complaints_iw=pd.merge(complaints_b,iwcodes, left_on='Practice_Code::Dental/GP Practice Code', right_on='Organisation Code')
gp[gp['Organisation Code'].isin(complaints_iw['Practice_Code::Dental/GP Practice Code'].tolist())].head()

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,...,Close Date,Status Code,Organisation Sub-Type code,Commissioner,Join Provider/Purchaser Date,Left Provider/Purchaser Date,Contact Telephone Number,Amended Record Indicator,Provider/Purchaser,Prescribing Setting
5255,J84003,VENTNOR MEDICAL CENTRE,Y57,Q70,VENTNOR MEDICAL CENTRE,3 ALBERT STREET,VENTNOR,ISLE OF WIGHT,None,PO38 1EZ,...,None,A,B,10L,2013-04-01 00:00:00,None,01983 857288,0,10L,4
5256,J84004,EAST COWES MEDICAL CENTRE,Y57,Q70,EAST COWES MEDICAL CENTRE,CHURCH PATH,EAST COWES,ISLE OF WIGHT,None,PO32 6RP,...,None,A,B,10L,2013-04-01 00:00:00,None,01983 284333,0,10L,4
5257,J84005,ESPLANADE SURGERY,Y57,Q70,THE ESPLANADE SURGERY,19 THE ESPLANADE,RYDE,ISLE OF WIGHT,None,PO33 2EH,...,None,A,B,10L,2013-04-01 00:00:00,None,01983 618388,0,10L,4
5258,J84007,ST.HELENS MEDICAL CENTRE,Y57,Q70,ST.HELENS MEDICAL CENTRE,UPPER GREEN ROAD,ST.HELENS,ISLE OF WIGHT,None,PO33 1UG,...,None,A,B,10L,2013-04-01 00:00:00,None,01983 871828,0,10L,4
5259,J84008,ARGYLL HOUSE,Y57,Q70,ARGYLL HOUSE,WEST STREET,RYDE,ISLE OF WIGHT,None,PO33 2QG,...,None,A,B,10L,2013-04-01 00:00:00,None,01983 562955,0,10L,4


In [122]:
import re

def epracurrDetails(code):
    return gp[gp['Organisation Code']==code].to_dict(orient='records')[0]

def getAddress(d):
    c=[x for x in d.keys() if 'address line' in x.lower()]
    c.append('Postcode')
    return [d[x] for x in c if ((d[x] is not None) and not pd.isnull(d[x]))]  

def receivedUpheld(d,k):
    rxd=d[k]['received'] if 'received' in d[k] else 0
    upheld=d[k]['upheld'] if 'upheld' in d[k] else 0
    return rxd,upheld

def receivedUpheldArea(d,k,hack2014=False):
    if hack2014: return receivedUpheld(d,k)
    rxd=d[k]['Complaints by area'] if 'Complaints by area' in d[k] else 0
    upheld=d[k]['Complaints upheld by area'] if 'Complaints upheld by area' in d[k] else 0
    return rxd,upheld

def receivedUpheldSubject(d,k,hack2014=False):
    if hack2014: return receivedUpheld(d,k)
    rxd=d[k]['Complaints by subject'] if 'Complaints by subject' in d[k] else 0
    upheld=d[k]['Complaints upheld by subject'] if 'Complaints upheld by subject' in d[k] else 0
    return rxd,upheld

def complaintReport(item,hack2014=False):
    details=epracurrDetails(item['Practice_Code::Dental/GP Practice Code'])
    txt='\n---------------\n'
    txt=txt+'Complaint report for: {name} ({code})\n'.format(name=details['Name'],
                                                      code=item['Practice_Code::Dental/GP Practice Code'])
    txt=txt+'\nAddress: {addr}.'.format(addr=', '.join(getAddress(details)))
    if ((details['Contact Telephone Number'] is not None) and not pd.isnull(details['Contact Telephone Number'])):
        txt=txt+'\nTelephone: {tel}'.format(tel=details['Contact Telephone Number'])
    txt=txt+'\n'
    
    if hack2014:
        complaints_received={'Complaint by subject':{},'Complaint by area':{}}
        complaints_upheld={'Complaint by subject':{},'Complaint by area':{}}
    else:
        complaints_received={'Complaints by subject':{},'Complaints by area':{}}
        complaints_upheld={'Complaints upheld by subject':{},'Complaints upheld by area':{}}
    
    complaints_by_area={}
    complaints_by_subject={}
    for col in item.index.values:
        if 'complaint' in col.lower() and item[col]>0:
            
            if hack2014:
                k=col.replace('\n','').split('Total number of written complaints ')

                if k[0]=='': k[0]='Total'
                k[0]=k[0].strip()
                k[1]=k[1]

                if 'received' in k[1]:
                    complaints_received[k[1].split('::')[1]][k[0]]=int(item[col])   
                elif 'upheld' in k[1]:
                    complaints_upheld[k[1].split('::')[1]][k[0]]=int(item[col])

                if 'area' in k[1]:
                    if k[0] not in complaints_by_area: complaints_by_area[k[0]]={}
                    complaints_by_area[k[0]][k[1].split('::')[0]]=int(item[col])
                elif 'subject' in k[1]:
                    if k[0] not in complaints_by_subject: complaints_by_subject[k[0]]={}
                    complaints_by_subject[k[0]][k[1].split('::')[0]]=int(item[col])
            else:
                if 'TOTAL' in col:
                    k=re.compile(r' SUBJECT TOTAL| TOTAL').split(col)
                    k[1]=k[1].lstrip('::')
                    complaints_received[k[1]][k[0]]=int(item[col])   
                elif 'UPHELD' in col:
                    k=re.compile(r' SUBJECT UPHELD| UPHELD').split(col)
                    k[1]=k[1].lstrip('::')
                    complaints_upheld[k[1]][k[0]]=int(item[col])

                if 'area' in col:
                    if k[0] not in complaints_by_area: complaints_by_area[k[0]]={}
                    complaints_by_area[k[0]][k[1].split('::')[0]]=int(item[col])
                elif 'subject' in col:
                    if k[0] not in complaints_by_subject: complaints_by_subject[k[0]]={}
                    complaints_by_subject[k[0]][k[1].split('::')[0]]=int(item[col])
                
    txt=txt+'\n'

    if complaints_by_area!={}:
        rxd,upheld=receivedUpheldArea(complaints_by_area,"Total",hack2014)
        txt=txt+'Complaints by area ({} received, of which {} upheld):'.format(rxd,upheld)
        for complaint in complaints_by_area:
            if complaint=='Total': continue
            rxd,upheld=receivedUpheldArea(complaints_by_area,complaint,hack2014)
            txt=txt+'\n    - {}: {} received, of which {} upheld.'.format(complaint,rxd,upheld)
        txt=txt+'\n\n'
    else: txt=txt+'No complaints by area.\n' 
        
    if complaints_by_subject!={}:
        rxd,upheld=receivedUpheldSubject(complaints_by_subject,"Total",hack2014)
        txt=txt+'Complaints by subject ({} received, of which {} upheld):'.format(rxd,upheld)
        for complaint in complaints_by_subject:
            if complaint=='Total': continue
            rxd,upheld=receivedUpheldSubject(complaints_by_subject,complaint,hack2014)
            txt=txt+'\n    - {}: {} received, of which {} upheld.'.format(complaint,rxd,upheld)
    else: txt=txt+'No complaints by subject.\n\n'      
    #print(txt)
    #print('\n---------------\n')
    
    return txt

reports=complaints_iw.apply(lambda x: complaintReport(x), axis=1 )

print('''
National Statistics Data on Written Complaints in the NHS - 2015-16 [NS]
Publication date: September 15, 2016
http://www.digital.nhs.uk/catalogue/PUB21533
''')

for report in reports:
    print(report)


National Statistics Data on Written Complaints in the NHS - 2015-16 [NS]
Publication date: September 15, 2016
http://www.digital.nhs.uk/catalogue/PUB21533


---------------
Complaint report for: ST.HELENS MEDICAL CENTRE (J84007)

Address: ST.HELENS MEDICAL CENTRE, UPPER GREEN ROAD, ST.HELENS, ISLE OF WIGHT, PO33 1UG.
Telephone: 01983 871828

Complaints by area (14 received, of which 14 upheld):
    - Practice admin: 5 received, of which 5 upheld.
    - Medical: 9 received, of which 9 upheld.

Complaints by subject (14 received, of which 14 upheld):
    - Communications/Attitude: 5 received, of which 5 upheld.
    - Practice/Surgery Management: 1 received, of which 1 upheld.
    - Practice admin: 2 received, of which 2 upheld.
    - Clinical: 6 received, of which 5 upheld.
    - Other: 0 received, of which 1 upheld.

---------------
Complaint report for: WEST WIGHT MEDICAL PRACTICE (J84019)

Address: BROOKSIDE HEALTH CENTRE, QUEENS ROAD, FRESHWATER, ISLE OF WIGHT, PO40 9DT.
Telephone: 

## Complaints Data - 2014 / 15

via: http://digital.nhs.uk/datacollections/ko41b
specifically: http://digital.nhs.uk/searchcatalogue?productid=18408

In [31]:
url='http://www.hscic.gov.uk/catalogue/PUB18021/data-writ-comp-nhs-2014-2015-csv.zip'
zip_2014_15=zipgrabber(url)
complaints_2014_15=pd.read_csv(zip_2014_15.open('Data on Written Complaints in the NHS 2014-15 KO41a csv.csv'))
complaints_2014_15.dropna(how='all',axis=1,inplace=True)
complaints_2014_15.head(3)

,Year,Area_Team_Code,Area_Team_Name,Org_Code,Org_Name,Hospital acute services: Inpatient total,Hospital acute services: Inpatient total upheld,Hospital acute services: Outpatient total,Hospital acute services: Outpatient total upheld,Hospital acute services: A&E total,...,Policy and commercial decisions of trusts total,Policy and commercial decisions of trusts total upheld,Code of openness - complaints total,Code of openness - complaints total upheld,Hotel services (including food) total,Hotel services (including food) total upheld,Other total.2,Other total upheld.2,TOTAL.2,TOTAL upheld.2
0,2014-15,Q45,"Durham, Darlington and Tees",00C,NHS Darlington CCG,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,4,0
1,2014-15,Q45,"Durham, Darlington and Tees",00D,"NHS Durham Dales, Easington and Sedgefield CCG",0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,3,2
2,2014-15,Q49,"Cumbria, Northumberland, Tyne and Wear",00F,NHS Gateshead CCG,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,4,2


In [83]:
get_metadata(zip_2014_15)

,Name,typ,meta
0,Year,(date time yyyy/yy),Year of collection
1,Return,(varchar),"Organisation type (Area Team, GP practice, Den..."
2,Area_Team_Code,(varchar),NHS England Area Team Code
3,Area_Team_Name,(varchar),NHS England Area Team Name
4,Practice_Code,(varchar),Dental/GP Practice Code
5,Medical Total number of written complaints rec...,(int),Complaint by area
6,Dental Total number of written complaints rece...,(int),Complaint by area
7,Practice Admin Total number of written complai...,(int),Complaint by area
8,Other Total number of written complaints received,(int),Complaint by area
9,Total number of written complaints received,(int),Complaint by area


In [114]:
complaints_b_2014_15=csvfileopener(zip_2014_15,'KO41b',True)
complaints_b_2014_15.head(3)

,Year::Year of collection,"Return::Organisation type (Area Team, GP practice, Dental practice) making return",Area_Team_Code::NHS England Area Team Code,Area_Team_Name::NHS England Area Team Name,Practice_Code::Dental/GP Practice Code,Medical Total number of written complaints received::Complaint by area,Dental Total number of written complaints received::Complaint by area,Practice Admin Total number of written complaints received::Complaint by area,Other Total number of written complaints received::Complaint by area,Total number of written complaints received::Complaint by area,...,Practice Admin Total number of written complaints upheld::Complaint by area,Other Total number of written complaints upheld::Complaint by area,Total number of written complaints upheld::Complaint by area,Communications/attitude Total number of written complaints upheld::Complaint by subject,Premises Total number of written complaints upheld::Complaint by subject,Practice/surgery management Total number of written complaints upheld::Complaint by subject,Practice administration Total number of written complaints upheld::Complaint by subject,Clinical Total number of written complaints upheld::Complaint by subject,Other Total number of written complaints upheld::Complaint by subject,Total number of written complaints upheld::Complaint by subject
0,2014-15,Dental,Q44,"Cheshire, Warrington and Wirral",V01008,0,0,0,0,0,...,0,0,0,0,0.0,0,0,0,0,0
1,2014-15,Dental,Q44,"Cheshire, Warrington and Wirral",V03463,0,1,0,0,1,...,0,0,1,0,0.0,0,0,1,0,1
2,2014-15,Dental,Q44,"Cheshire, Warrington and Wirral",V04661,0,1,0,0,1,...,0,0,0,0,0.0,0,0,0,0,0


In [125]:
#join complaints parent codes
parentGP=gp[['Organisation Code','Commissioner']]
parentGP.columns=['Organisation Code','Parent Organisation Code']
parentDentist=dentists[['Organisation Code','Parent Organisation Code']]
codeparent=pd.concat([parentGP,parentDentist])
codeparent.head(3)

,Organisation Code,Parent Organisation Code
0,A81001,00K
1,A81002,00K
2,A81003,00K


In [126]:
iwcodes=codeparent[(codeparent['Parent Organisation Code']=='10L')]
complaints_iw=pd.merge(complaints_b_2014_15,iwcodes, left_on='Practice_Code::Dental/GP Practice Code', right_on='Organisation Code')
gp[gp['Organisation Code'].isin(complaints_iw['Practice_Code::Dental/GP Practice Code'].tolist())].head()

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,...,Close Date,Status Code,Organisation Sub-Type code,Commissioner,Join Provider/Purchaser Date,Left Provider/Purchaser Date,Contact Telephone Number,Amended Record Indicator,Provider/Purchaser,Prescribing Setting
5255,J84003,VENTNOR MEDICAL CENTRE,Y57,Q70,VENTNOR MEDICAL CENTRE,3 ALBERT STREET,VENTNOR,ISLE OF WIGHT,None,PO38 1EZ,...,None,A,B,10L,2013-04-01 00:00:00,None,01983 857288,0,10L,4
5256,J84004,EAST COWES MEDICAL CENTRE,Y57,Q70,EAST COWES MEDICAL CENTRE,CHURCH PATH,EAST COWES,ISLE OF WIGHT,None,PO32 6RP,...,None,A,B,10L,2013-04-01 00:00:00,None,01983 284333,0,10L,4
5257,J84005,ESPLANADE SURGERY,Y57,Q70,THE ESPLANADE SURGERY,19 THE ESPLANADE,RYDE,ISLE OF WIGHT,None,PO33 2EH,...,None,A,B,10L,2013-04-01 00:00:00,None,01983 618388,0,10L,4
5258,J84007,ST.HELENS MEDICAL CENTRE,Y57,Q70,ST.HELENS MEDICAL CENTRE,UPPER GREEN ROAD,ST.HELENS,ISLE OF WIGHT,None,PO33 1UG,...,None,A,B,10L,2013-04-01 00:00:00,None,01983 871828,0,10L,4
5259,J84008,ARGYLL HOUSE,Y57,Q70,ARGYLL HOUSE,WEST STREET,RYDE,ISLE OF WIGHT,None,PO33 2QG,...,None,A,B,10L,2013-04-01 00:00:00,None,01983 562955,0,10L,4


In [127]:
reports=complaints_iw.apply(lambda x: complaintReport(x,hack2014=True), axis=1 )

print('''
National Statistics Data on Written Complaints in the NHS - 2014-15 [NS]
Publication date: ??, 2015
http://www.digital.nhs.uk/catalogue/PUB21533
''')

for report in reports:
    print(report)


National Statistics Data on Written Complaints in the NHS - 2014-15 [NS]
Publication date: ??, 2015
http://www.digital.nhs.uk/catalogue/PUB21533


---------------
Complaint report for: VENTNOR MEDICAL CENTRE (J84003)

Address: VENTNOR MEDICAL CENTRE, 3 ALBERT STREET, VENTNOR, ISLE OF WIGHT, PO38 1EZ.
Telephone: 01983 857288

Complaints by area (1 received, of which 0 upheld):
    - Practice Admin: 1 received, of which 0 upheld.

Complaints by subject (3 received, of which 0 upheld):
    - Clinical: 1 received, of which 0 upheld.
    - Communications/attitude: 1 received, of which 0 upheld.
    - Practice administration: 1 received, of which 0 upheld.

---------------
Complaint report for: EAST COWES MEDICAL CENTRE (J84004)

Address: EAST COWES MEDICAL CENTRE, CHURCH PATH, EAST COWES, ISLE OF WIGHT, PO32 6RP.
Telephone: 01983 284333

Complaints by area (11 received, of which 11 upheld):
    - Practice Admin: 6 received, of which 6 upheld.
    - Medical: 4 received, of which 4 upheld.
 